In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [4]:
nltk.download('punkt')

NameError: ignored

In [0]:
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

def form_sentence(tweet):
    tweet_blob = TextBlob(tweet)
    return ' '.join(tweet_blob.words)

def no_user_alpha(tweet):
    tweet_list = [ele for ele in tweet.split() if ele != 'user']
    clean_tokens = [t for t in tweet_list if re.match(r'[^\W\d]*$', t)]
    clean_s = ' '.join(clean_tokens)
    clean_mess = [word for word in clean_s.split() if word.lower() not in stopwords.words('english')]
    return clean_mess

def normalization(tweet_list):
    lem = WordNetLemmatizer()
    normalized_tweet = []
    for word in tweet_list:
        normalized_text = lem.lemmatize(word,'v')
        normalized_tweet.append(normalized_text)
    return normalized_tweet
    
def preprocessTweets(tweetlist):
    processedTweets=[]
    for tweet in tweetlist:
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = form_sentence(tweet)
        tweet = no_user_alpha(tweet)
        tweet = normalization(tweet)
        processedTweets.append(tweet)
    return processedTweets

In [0]:
df = pd.read_csv("./train.csv")
train_data = df['text'].to_numpy()
train_label = df['labels'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(train_data, train_label, test_size=0.3, random_state=42)
X_train = preprocessTweets(X_train)
X_test = preprocessTweets(X_test)


**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt/PY3/english.pickle

  Searched in:
    - '/home/akshay/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************



MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


In [0]:
Train_X_Tfidf = []
count = 0
num_view_classfier = 10
tfidf_vectorizer = [TfidfVectorizer(use_idf=True, ngram_range=(i+1,i+1)) for i in range(num_view_classfier)]
for i in range(num_view_classfier):
    fit_data=tfidf_vectorizer[i].fit(X_train)#_transform(Train_X)
    Train_X_Tfidf.append(fit_data.transform(X_train))
    count+= np.shape(Train_X_Tfidf[i])[1]

In [0]:
view_svm = [svm.SVC(C=0.1, kernel='linear', probability=True) for i in range(num_view_classfier)]
decision_scores = []
for i in range(num_view_classfier):
    view_svm[i].fit(Train_X_Tfidf[i],y_train)
    decision_scores.append(view_svm[i].decision_function(Train_X_Tfidf[i]))
    decision_scores.append(1-decision_scores[2*i])

In [0]:
train_meta = np.transpose(decision_scores)
meta_svm = svm.SVC(C=0.1, kernel='linear')
meta_svm.fit(train_meta, y_train)

SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
Test_X_Tfidf = []
for i in range(num_view_classfier):
    Test_X_Tfidf.append(tfidf_vectorizer[i].transform(X_test))

In [0]:
test_decision = []
for i in range(num_view_classfier):
    test_decision.append(view_svm[i].decision_function(Test_X_Tfidf[i]))
    test_decision.append(1-test_decision[2*i])

In [0]:
test_meta = np.transpose(test_decision)
acc= meta_svm.score(test_meta, y_test)
f1 = f1_score(y_test, meta_svm.predict(test_meta), average='weighted')
print(acc, f1)

0.6430379746835443 0.585635325876141


In [0]:
from sklearn.linear_model import LogisticRegression

lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.5 # if prediction is greater than or equal to 0.5 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

In [0]:
RFclf = RandomForestClassifier(n_jobs=2, random_state=0)
RFclf.fit(training_set_X, training_set_y)

prediction = clf.predict(val_set_X)
correct += np.sum(prediction == val_set_y)